In [13]:
import keras
from keras import layers
import tensorflow as tf
from keras import applications, models, layers

import os
print(os.listdir('./data'))


train_dataset = keras.utils.image_dataset_from_directory(
    './data',                       # Path to your dataset
    labels='inferred',             # Infer labels from folder names
    label_mode='int',              # Binary or integer labels
    batch_size=32,                 # Number of images per batch
    image_size=(256, 256),         # Resize all images to 256x256
    shuffle=True,                  # Shuffle the dataset
    validation_split=0.5,          # Reserve 20% for validation
    subset='training',             # Use this for training data
    seed=123                       # Seed to ensure consistent split
)

validation_dataset = keras.utils.image_dataset_from_directory(
    './data',                       # Same directory for validation data
    labels='inferred',            # Infer labels
    label_mode='int',             # Binary or integer labels
    batch_size=32,                # Batch size
    image_size=(256, 256),        # Resize all images
    shuffle=True,                 # Shuffle the dataset
    validation_split=0.2,         # 20% of the data for validation
    subset='validation',          # Use this for validation data
    seed=123                      # Seed to ensure consistent split
)

# Normalize the pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optional: Prefetch to optimize performance during training
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Build the model using MobileNetV2 as the base
base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model layers

# Build the custom model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification (fire risk vs no fire risk)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# Unfreeze some layers of the base model and fine-tune
base_model.trainable = True
# Refreeze initial layers if needed (optional)
for layer in base_model.layers[:-40]:
    layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training (fine-tuning)
fine_tune_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

# Save the model
model.save('fire_risk_model.h5')

['fire', 'nofire']
Found 4 files belonging to 2 classes.
Using 2 files for training.
Found 4 files belonging to 2 classes.
Using 0 files for validation.
Epoch 1/10


/var/folders/4s/0shkycv9613d_rw6h0lfrjqw0000gn/T/ipykernel_57010/675304008.py:46: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.5746 - val_accuracy: 0.7500 - val_loss: 0.4148
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 1.0000 - loss: 0.0713 - val_accuracy: 0.7500 - val_loss: 0.3526
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0150 - val_accuracy: 0.7500 - val_loss: 0.3344
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 1.0000 - loss: 0.0050 - val_accuracy: 0.7500 - val_loss: 0.3253
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.7500 - val_loss: 0.3167
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.7500 - val_loss: 0.3068
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 6.6494e-04 - val_accuracy: 0.7500 - val_loss: 0.2969
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 4.1590e-04 - val_accuracy: 0.7500 - val_loss: 0.2869
Epoc

In [14]:
train_loss, train_accuracy = model.evaluate(train_dataset)
print(f'Training Loss: {train_loss}')
print(f'Training Accuracy: {train_accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 1.1816e-04
Training Loss: 0.00011816388723673299
Training Accuracy: 1.0
